In [1]:
!pip install datasets torch_optimizer lion_pytorch clang_repl_kernel --break-system-packages
!pip install --upgrade clang-repl-kernel  --break-system-packages
#!apt update
#!apt-get install libc++1 libc++abi1

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/torchtext-0.18.0a0+9bed85d-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/torchaudio-2.6.0a0+d883142-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg i

In [2]:
from ClangReplInterface import ClangReplInterface
clang_repl = ClangReplInterface()


Lin64.zip: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 350M/350M [00:02<00:00, 118MB/s]


Download completed: /usr/local/lib/python3.12/dist-packages/clang_repl_kernel/clang/Lin64/Lin64.zip
Extracting /usr/local/lib/python3.12/dist-packages/clang_repl_kernel/clang/Lin64/Lin64.zip to /usr/local/lib/python3.12/dist-packages/clang_repl_kernel/clang/Lin64
Reading zip file... Please wait.


Extracting: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3245/3245 [00:04<00:00, 675.35file/s]


In [3]:
!rm -rf "runs/starcoder2_reasoning"

In [4]:
#!rm -rf "saved_models/reasoning"

In [5]:
import os
import copy
import re
import json
import torch
import gc
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
)
from transformers.optimization import Adafactor
from datasets import Dataset
import signal
from tqdm import tqdm

# For hyperparameter optimization
import optuna
import pickle
from Config import SimpleConfig
from ClangReplInterface import ClangReplInterface
import warnings

warnings.filterwarnings(
    "ignore",
    category=FutureWarning
)

ref_checkpoint_path = "./saved_models/sample/checkpoint.pt"
last_checkpoint_path = "./saved_models/reasoning/checkpoint.pt"
checkpoint_dir_pre = "./saved_models/reasoning/epoch_"

test_target_object_file = "./saved_models/ReasoningTestTarget_prompt.json"

config = SimpleConfig()

max_length = 512


/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:106: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


INFO:datasets:PyTorch version 2.7.0a0+ecf3bae40a.nv25.2 available.
INFO:datasets:Polars version 1.14.0 available.


In [ ]:

# num_iterations=1, num_steps=500, batch_size=4, num_generations=4, max_completion_length=128, kl=0.1,
# learning_rate=5e-6, mu=3, epsilon=0.2,
#
# lr: 7.205691481165551e-05 kl_lambda: 0.2654706177039008 epsilon: 0.019437902361559744 num_grpo: 1
# lr: 1.1111588431283189e-06 kl_lambda: 0.15842765249477542 epsilon: 0.11144786260484413 num_grpo: 3
MAX_REWARD=4.4 

log_content = False
log_step = False
log_memory = False

full_layer_train=True

is_finding_opt = True
if not is_finding_opt:
    num_epochs = 200
    lr = 1.1111588431283189e-06
    kl_lambda = 0.15842765249477542
    epsilon = 0.11144786260484413
    num_grpo = 1
    save_epochs = 10
    warming_up_step= 10
    gradient_accumulation_step = 1
    temperature = 1.0
    
group_size = 4
batch_size = 1 # 7

def pad_to_match(tensor_a, tensor_b, padding_value=0):
    # Determine the current sequence lengths
    seq_len_a = tensor_a.size(1)
    seq_len_b = tensor_b.size(1)

    if seq_len_a > seq_len_b:
        max_seq_len = max(seq_len_a, seq_len_b)
    
        # Define padding function
        def pad_tensor(tensor, target_length):
            pad_length = target_length - tensor.size(1)
            if pad_length > 0:
                padding = (0, 0) * (tensor.dim() - 2) + (0, pad_length)
                tensor = F.pad(tensor, padding, value=padding_value)
            return tensor
    
        # Pad both tensors to the maximum sequence length
        tensor_a_padded = pad_tensor(tensor_a, max_seq_len)
        tensor_b_padded = pad_tensor(tensor_b, max_seq_len)
    else:
        tensor_b_padded = tensor_b[:, :seq_len_a]
        tensor_a_padded = tensor_a

    return tensor_a_padded, tensor_b_padded


def reward_atag(front, end, response):
    tag_len = len(front + end)
    start = response.find(front + end)
    end = response.find(front + '/' + end)
    reward = 0
    if start != -1: reward += 0.1
    if end != -1: reward += 0.1
    
    if start + tag_len < end:
        if len(response[start + tag_len:end].strip()) > 1:
            reward += 0.1
    return reward


def remove_comments(code: str):
    pattern = re.compile(r'//.*?$|/\*.*?\*/', re.DOTALL | re.MULTILINE)
    return re.sub(pattern, '', code)


def find_all_tag_indexes(text, tag):
    """Return a list of starting indexes where the tag occurs in the text."""
    indexes = []
    start = 0
    while True:
        idx = text.find(tag, start)
        if idx == -1:
            break
        indexes.append(idx)
        start = idx + len(tag)
    return indexes


def get_tag_start_end(idx, starts, ends, tag, full_text):
    start = starts[idx]+len(tag)
    end = ends[idx]
    return full_text[start: end].strip()
    

def reward_correct(full_text):
    # handle only first answer
    reward = 0.0
    test_target_open = find_all_tag_indexes(full_text, "<Test Target>")
    test_target_close = find_all_tag_indexes(full_text, "</Test Target>")
    clang_repl_open = find_all_tag_indexes(full_text, "<Clang-repl Test>")
    clang_repl_close = find_all_tag_indexes(full_text, "</Clang-repl Test>")
    if len(test_target_open) == 0 or len(test_target_close) == 0 or len(clang_repl_open) == 0 or len(clang_repl_close) == 0:
        return reward, '<Test Target> or <Clang-repl Test> not found'
    if len(test_target_open) != len(test_target_close) or len(clang_repl_open) != len(clang_repl_close):
        return reward , '<Test Target> or <Clang-repl Test> pair not match'
    if not all(x < y for x, y in zip(test_target_open, test_target_close)):
        return reward, '<Test Target> not closed properly'
    if not all(x < y for x, y in zip(clang_repl_open, clang_repl_close)):
        return reward, '<Clang-repl Test> not closed properly' 
    target_text = get_tag_start_end(-1, test_target_open, test_target_close, "<Test Target>", full_text)
    target_text = remove_comments(target_text)
    target_text = ">>> "+target_text.replace('\n', '')

    for idx in range(len(clang_repl_open)):
        clang_repl_test = get_tag_start_end(idx, clang_repl_open, clang_repl_close, "<Clang-repl Test>", full_text)
        clang_repl = ClangReplInterface()
        test_case_with_target = target_text+'\n'+clang_repl_test
        #print(test_case_with_target)
        result, response = clang_repl.run_verify(test_case_with_target)
        reward = 0.0
        if result == 'ok':
            reward = 2.0
        elif result == 'fail':
            reward = 1.0
        elif result == 'error':
            reward = 0.0
        else:
            assert False
        return reward, response
    else:
        return reward, ''

def _reward(full_text):
    # https://blog.gopenai.com/coding-grpo-from-scratch-a-guide-to-distributed-implementation-with-qwen2-5-1-5b-instruct-59b34227edacabs
    format_rewards = []
    rewards = []
    responses = []
    for response_full in full_text:
        need_more_test_idx = response_full.find('<Need More Test')
        response = response_full if need_more_test_idx == -1 else response_full[:need_more_test_idx]
        score = 0.0
        score += reward_atag("<", "Test Object>", response) # 0.3
        score += reward_atag("<", "Input Data>", response) # 0.3
        score += reward_atag("<", "Expected Output>", response) # 0.3
        score += reward_atag("<", "Clang-repl Test>", response) # 0.3
        score += reward_atag("[", "REASON]", response) * 2 # 0.3*2
        score += reward_atag("[", "ANSWER]", response) * 2 # 0.3*2
        score = score / 10 # 0.24 *10 = 2.4
        format_rewards.append(score)
        correct_reward, response = reward_correct(response)
        rewards.append(score + correct_reward)
        responses.append(response)

    return rewards, responses

# Custom exception for timeouts.
class TimeoutException(Exception):
    pass

def _handle_timeout(signum, frame):
    raise TimeoutException("Function call timed out.")
    
def reward(full_text, writer=None, global_step=None):
    # Set the signal handler and alarm.
    signal.signal(signal.SIGALRM, _handle_timeout)
    signal.alarm(5)  # Set timeout to 5 seconds.
    try:
        results, responses = _reward(full_text)
    except TimeoutException:
        # Log the timeout event to TensorBoard under a specific tag if a writer is provided.
        if writer is not None:
            writer.add_text("TimeoutEvents/RewardFunction", f"Timeout occurred at global step {global_step}", global_step=global_step)
        results = [0]*len(full_text)
        responses = ["reward() timeout"]*len(full_text)
    finally:
        # Cancel the alarm.
        signal.alarm(0)
    return results, responses

def object_hiper_param(trial):
    # Shortened training for demonstration:
    num_epochs = 1  # 2   # or 2–3, to save time during hyperparameter search

    # Sample hyperparameters
    lr = trial.suggest_float("lr", 1e-6, 1e-3, log=True) # 5e-6
    kl_lambda = trial.suggest_float("kl_lambda", 0.001, 0.08) # 0.04 from
    epsilon = trial.suggest_float("epsilon", 0.01, 0.2) # 0.1
    num_grpo = trial.suggest_int("num_grpo", 1, 3, step=1) # 1
    warming_up_step= trial.suggest_int("warming_up_step", 1, 1, step=1)
    gradient_accumulation_step = trial.suggest_int("gradient_accumulation_step", 1, 5, step=1)
    temperature = trial.suggest_float("temperature", 0.5, 1.2) # 1.0

    return num_epochs, lr, kl_lambda, epsilon, num_grpo, warming_up_step, gradient_accumulation_step, temperature

def list_cuda_tensors():
    cuda_tensors = []
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) and obj.is_cuda:
                cuda_tensors.append(obj)
        except Exception:
            pass  # Some objects might not have the attributes we need.
    return cuda_tensors

previous_tensor_info = {}
def print_memory(tag):
    global previous_tensor_info
    if not log_memory:
        return
    # Make sure you have a GPU device available.
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Print current allocated and reserved memory in MB:
    allocated = torch.cuda.memory_allocated(device) / (1024 ** 2)
    reserved = torch.cuda.memory_reserved(device) / (1024 ** 2)
    print(tag)
    print(f"Memory allocated: {allocated:.2f} MB")
    print(f"Memory reserved: {reserved:.2f} MB")

    
    if False:
        print(torch.cuda.memory_summary(device=None, abbreviated=False))
        
    if True:
        # List all CUDA tensors and collect their details in a dictionary.
        cuda_tensors = list_cuda_tensors()
        current_tensor_info = {id(tensor): (tensor.shape, tensor.device) for tensor in cuda_tensors}
        
        # Determine new and deleted tensor IDs.
        current_ids = set(current_tensor_info.keys())
        previous_ids = set(previous_tensor_info.keys())

        print("Total Tensors:", len(current_ids), ", Changes:", len(current_ids)-len(previous_ids))

        if False:
            # Determine new tensors since the last call.
            new_tensor_ids = current_tensor_ids - previous_tensor_ids
            deleted_tensor_ids = previous_ids - current_ids
            if new_tensor_ids:
                print("New CUDA tensors created since the last call:")
                for tid in new_tensor_ids:
                    shape, dev = current_tensor_info[tid]
                    print(f"Tensor id: {tid} | Shape: {shape} | Device: {dev}")
    
            if deleted_tensor_ids:
                print("Deleted CUDA tensors since the last call:")
                for tid in deleted_tensor_ids:
                    shape, dev = previous_tensor_info[tid]
                    print(f"Tensor id: {tid} | Shape: {shape} | Device: {dev}")
        previous_tensor_info = current_tensor_info.copy()

def print_step(tag, main_step=False):
    if log_memory:
        print_memory(tag)
    else:
        if log_step or main_step:
            print(tag)

def check_optimizer_duplicates(optimizer):
    seen_ids = set()
    duplicates = []
    for group in optimizer.param_groups:
        for param in group['params']:
            pid = id(param)
            if pid in seen_ids:
                duplicates.append(param)
            else:
                seen_ids.add(pid)
    return duplicates

def samping(model, tokenizer, device, epoch, writer, sample_prompt, expected):
    # Include attention_mask in the tokenization
    sample_prompt = f"### Instruction\n\n{sample_prompt}\n\n### Response"
    inputs = tokenizer(sample_prompt, return_tensors="pt", return_attention_mask=True)
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)

    # Pass the attention_mask and explicitly set pad_token_id to eos_token_id for reliable generation
    generated_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=20,
        pad_token_id=tokenizer.eos_token_id
    )
    sample_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    sample_text = sample_text.strip()
    if (log_content):print(f"Sample Output (Epoch {epoch + 1}): {sample_text}")
    if (log_content):print("Expected:", expected)
    writer.add_text("Sample Output", f"Epoch {epoch + 1}: {sample_text}", epoch)



def selective_log_softmax(logits, input_ids, tokenizer):
    # Ensure input_ids are on the same device as logits
    if input_ids.device != logits.device:
        input_ids = input_ids.to(logits.device)

    log_probs = nn.functional.log_softmax(logits, dim=-1)
    if input_ids.size(1) > log_probs.size(1):
        input_ids = input_ids[:, :log_probs.size(1)]

    # Gather log probabilities corresponding to input_ids
    selected_log_probs = log_probs.gather(dim=-1, index=input_ids.unsqueeze(-1)).squeeze(-1)

    if (log_content):
        input_text = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
        print("Input Texts:")
        for text in input_text:
            print(text)
        logits_ids = logits.argmax(dim=-1)
        logit_text = tokenizer.batch_decode(logits_ids, skip_special_tokens=True)
        print("\nLogit Texts:")
        for text in logit_text:
            print(text)

    return selected_log_probs

def add_front_transformer_block(self, copy_weights: bool = True):
    # Retrieve the current first transformer block.
    layer_index = 1
    original_first_block = self.model.layers[layer_index]

    # Create a new block.
    new_block = copy.deepcopy(original_first_block) if copy_weights else type(original_first_block)()

    self.model.layers.insert(layer_index, new_block)

    self.config.num_hidden_layers += 1


def get_layer_params(self, layer_index: int = 0):
    if full_layer_train:
        return self.model.parameters()
    else:
        first_params = list(self.model.layers[0].parameters())
        sec_params = list(self.model.layers[0].parameters())
        # last_params = list(self.model.layers[-1].parameters())
        return first_params + sec_params  # + last_params


def cut_tensors_by_min(a: torch.Tensor, b: torch.Tensor, dim: int):
    """
    Cut tensors `a` and `b` along a specified dimension to the smaller length 
    between them along that dimension.

    Args:
        a (torch.Tensor): A 2D or 3D tensor.
        b (torch.Tensor): A 2D or 3D tensor.
        dim (int): The dimension along which to slice.

    Returns:
        Tuple[torch.Tensor, torch.Tensor]: Sliced versions of `a` and `b`.
    """
    assert a.dim() > dim and b.dim() > dim, "Specified dim exceeds tensor rank"

    min_length = min(a.size(dim), b.size(dim))
    a_cut = torch.narrow(a, dim, 0, min_length)
    b_cut = torch.narrow(b, dim, 0, min_length)
    return a_cut, b_cut

def generate_ids(model, batch, tokenizer, temperature):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    eos_token_id = tokenizer.eos_token_id

    # Determine prompt length for each example in the batch based on the first occurrence of EOS.
    prompt_lengths = []
    for i in range(input_ids.size(0)):
        seq = input_ids[i]
        # Find indices where the token equals the eos_token_id.
        eos_positions = (seq == eos_token_id).nonzero(as_tuple=True)[0]
        # If there's at least one occurrence, use its index + 1 (if you want to include the EOS in the prompt).
        # Otherwise, fallback to the full sequence length.
        if eos_positions.numel() > 0:
            first_eos = eos_positions[0].item() + 1
        else:
            first_eos = seq.size(0)
        prompt_lengths.append(first_eos)

    print_memory("Prompt lengths per batch element: " + str(prompt_lengths))

    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,  # assuming max_length is defined globally
        temperature=temperature,
        do_sample=True,
        eos_token_id=eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        output_scores=True,
        return_dict_in_generate=True,
        early_stop=False
    )
    generated_ids = output.sequences.detach()
    print_memory("generated_ids.shape[-1]: " + str(generated_ids.shape[-1]))
    return generated_ids, prompt_lengths


def compute_logits(model, generated_ids, prompt_lengths, tokenizer):
    generated_ids_mask = (generated_ids != tokenizer.pad_token_id).long().to(generated_ids.device)
    logits = model(input_ids=generated_ids, attention_mask=generated_ids_mask, early_stop=False).logits
    
    completion_ids_list = []
    completion_logits_list = []
    batch_size = generated_ids.size(0)
    
    for i in range(batch_size):
        p_len = prompt_lengths[i]
        
        comp_ids = generated_ids[i, p_len:].detach()
        comp_logits = logits[i, p_len-1:, :]
        
        completion_ids_list.append(comp_ids)
        completion_logits_list.append(comp_logits)
    
    # Determine the minimum length across the batch for completion tokens.
    min_length_ids = min(ids.size(0) for ids in completion_ids_list)
    min_length_logits = min(lgts.size(0) for lgts in completion_logits_list)
    min_length = min(min_length_ids, min_length_logits)
    
    # Truncate all tensors to the minimum length and stack them.
    completion_ids_truncated = torch.stack([ids[:min_length] for ids in completion_ids_list])
    completion_logits_truncated = torch.stack([lgts[:min_length] for lgts in completion_logits_list])
    
    return completion_logits_truncated, completion_ids_truncated
    
def process_generated_ids(generated_ids, eos_token_id):
    processed_ids = []
    for seq in generated_ids:
        if eos_token_id in seq:
            # Truncate the sequence at the first occurrence of the EOS token
            first_eos_index = seq.index(eos_token_id)
            processed_ids.append(seq[:first_eos_index])
        else:
            processed_ids.append(seq)
    return processed_ids

# ------------------------------------------------
# Load Q&A from JSON file (manual_data_set/QA.json)
# and create a list of {"content": "..."}
# ------------------------------------------------
def load_qa_dataset(json_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    train_examples = []
    for item in data:
        q = item.get("Q", "")
        a = item.get("A", "")
        content = f"### Instruction\n\n{q}\n### Response\n\n{a}\n"
        train_examples.append({"content": content + "<|endoftext|>"})
    return train_examples


def load_sample_dataset(pk_file):
    with open(config.dataset_file, "rb") as f:
        global_samples = pickle.load(f)
        sample_dataset = []
        for sample in global_samples:
            sample_dataset.append({"content": sample + "<|endoftext|>"})
        return sample_dataset

def get_test_target_content(full_text):
    test_target_open = find_all_tag_indexes(full_text, "<Test Target>")
    test_target_close = find_all_tag_indexes(full_text, "</Test Target>")
    target_text = get_tag_start_end(-1, test_target_open, test_target_close, "<Test Target>", full_text)
    return target_text

def load_reasoning_dataset(test_target_object_file):
    with open(test_target_object_file, 'r', encoding='utf-8') as file:
        data = json.load(file)
        train = []
        val = []
        categories = set()
        data_dic = {}
        for item in data:
            categories.add(item['category'])
        for cat in categories:
            data_dic[cat] = []
        for item in data:
            data_dic[item['category']].append(item['content'])
        for cat in categories:
            for idx, item in enumerate(data_dic[cat]):
                content = f"### Instruction\n\nn<Test Target>\n{get_test_target_content(item)}\n</Test Target>\nWrtie a Clang-repl Test\n### Response\n"
                if idx >=10:
                    val.append({"content":content})
                else:
                    train.append({"content":content})

        return train, val

# Provide the path to your Q&A JSON file
qa_json_path = "manual_data_set/QA.json"
train_data_prompt = load_qa_dataset(qa_json_path)
train_data_sample = load_sample_dataset(config.dataset_file)
train_data = train_data_prompt + (train_data_sample * 10)

test_target_object_file = "manual_data_set/ReasoningTestTarget.json"
reasoning_dataset, val_reasoning_dataset = load_reasoning_dataset(test_target_object_file)
reasoning_dataset = reasoning_dataset[:10]
val_reasoning_dataset = val_reasoning_dataset[:5]

# Create a Hugging Face Dataset from the list
train_dataset = Dataset.from_list(reasoning_dataset)
val_train_dataset = Dataset.from_list(val_reasoning_dataset)
# ------------------------------------------------
# Define Tokenization
# ------------------------------------------------
model_id = "bigcode/starcoder2-3b"
# Load tokenizer from saved directory if exists; otherwise, load from pretrained.
tokenizer_save_dir = "./saved_models/tokenizer"
if os.path.exists(tokenizer_save_dir):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_save_dir)
    tokenizer.padding_side = 'left'
    if log_step: print("Loaded tokenizer from saved checkpoint.")
else:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.padding_side = 'left'
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    return tokenizer(
        examples["content"],
        truncation=True,
        max_length=max_length,
        #padding="max_length"
    )

if log_step: print("eos: ", tokenizer.eos_token, tokenizer.eos_token_id)

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["content"])
tokenized_dataset.set_format("torch")

val_tokenized_dataset = val_train_dataset.map(tokenize_function, batched=True)
val_tokenized_dataset = val_tokenized_dataset.remove_columns(["content"])
val_tokenized_dataset.set_format("torch")

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


# ------------------------------------------------
# Define Training Function
# ------------------------------------------------
def run(model, old_model, ref_model, dataloader, optimizer, device, tokenizer,
        group_size, epsilon, kl_lambda, scaler, writer, global_step, log_group,
        warm_up_step, gradient_accumulation_step, is_validation=False, temperature=1.0):
    running_loss = 0.0
    print_memory("_.1. run() enter")
    # For accumulation mode, ensure gradients are zeroed at the start.
    if not is_validation and global_step >= warm_up_step:
        optimizer.zero_grad()

    for step, batch in enumerate(tqdm(dataloader, total=len(dataloader)), start=1):
        print_step(f"Processing batch {step}/{len(dataloader)}: Start Loop")
        # Move batch to device and expand the tensors.
        batch = {k: v.to(device) for k, v in batch.items()}
        input_ids = batch['input_ids'].repeat_interleave(group_size, dim=0)
        attention_mask = batch['attention_mask'].repeat_interleave(group_size, dim=0)
        batch['input_ids'] = input_ids
        batch['attention_mask'] = attention_mask

        with torch.amp.autocast(device_type='cuda'):
            # 1. Model forward pass for generation.
            print_step("1. Model train")
            with torch.no_grad():
                generated_ids, prompt_lengths = generate_ids(old_model, batch, tokenizer, temperature)
            completion_logits, completion_ids = compute_logits(model, generated_ids, prompt_lengths, tokenizer)
            full_ids = process_generated_ids(generated_ids, tokenizer.eos_token_id)
            
            # 2. Run legacy models (old and reference models).
            print_step("2. Legacy Models Run")
            with torch.no_grad():
                old_completion_logits, old_completion_ids = compute_logits(old_model, generated_ids, prompt_lengths, tokenizer)
                ref_completion_logits, ref_completion_ids = compute_logits(ref_model, generated_ids, prompt_lengths, tokenizer)
            
            with torch.no_grad():
                model_log_logits = selective_log_softmax(completion_logits, completion_ids, tokenizer)
                old_model_log_logits = selective_log_softmax(old_completion_logits, old_completion_ids, tokenizer)
                model_log_logits, old_model_log_logits = pad_to_match(model_log_logits, old_model_log_logits)
                probability_ratio = torch.exp(model_log_logits - old_model_log_logits)
                # Remove legacy model intermediates (no longer needed)
                old_completion_logits = None
                old_completion_ids = None
                ref_completion_ids = None
                model_log_logits = None
                old_model_log_logits = None

            # 3. Calculate rewards.
            print_step("3. Reward calc")
            full_text_lists = tokenizer.batch_decode(full_ids, skip_special_tokens=True)
            rewards, responses = reward(full_text_lists, writer, global_step)
            # Release unused tensors from generation.
            full_ids = None
            full_text_lists = None

            # Convert rewards to tensor
            advantages = torch.tensor(rewards, dtype=torch.float32, device=device)
            
            # Reshape rewards to match batch structure (assuming same shape as in first implementation)
            batch_size = advantages.size(0) // group_size
            advantages = advantages.view(batch_size, group_size)
            
            # Calculate mean and std per batch (along dim=1) and repeat to match original size
            mean_rewards = advantages.mean(dim=1).repeat_interleave(group_size)
            std_rewards = advantages.std(dim=1).repeat_interleave(group_size)
            print("std_rewards:", std_rewards[0].item(), "Rewards:", rewards)

            # Reshape back to original form
            advantages = advantages.view(-1)
            
            _A_hat = (advantages - mean_rewards) / std_rewards
            A_hat = _A_hat.unsqueeze(1)
            
            # Clear rewards intermediates.
            advantages = None
            _A_hat = None

            # 4. Loss Calculation.
            print_step("4. Loss Calc")            
            # PPO objective calculations.
            unclipped_objective = probability_ratio * A_hat
            clipped_ratio = torch.clamp(probability_ratio, 1 - epsilon, 1 + epsilon)
            clipped_objective = clipped_ratio * A_hat
            
            # Log objective stats.
            surrogate_loss = torch.min(unclipped_objective, clipped_objective)  # shape: [batch, seq_length]
            
            # Calculate token-level log probabilities.
            model_log_probs = F.log_softmax(completion_logits, dim=-1)   # log probs for model outputs
            ref_log_probs = F.log_softmax(ref_completion_logits, dim=-1)   # log probs for reference outputs
            
            # Make sure both tensors have the same shape.
            ref_log_probs, model_log_probs = pad_to_match(ref_log_probs, model_log_probs)
            
            # Compute token-level KL divergence.
            kl_div = torch.exp(ref_log_probs - model_log_probs) - (ref_log_probs - model_log_probs) - 1
            kl_div = kl_div.sum(dim=-1) 
            
            # Create a mask for non-padding tokens.
            completion_mask = (completion_ids != tokenizer.pad_token_id).float()  # shape: [batch, seq_length]
            
            # Assume beta is a scaling factor for the KL term (e.g., beta = kl_lambda).
            per_token_loss = surrogate_loss - kl_lambda * kl_div
            
            # Final loss: sum the per-token losses over tokens, normalize by the number of non-padding tokens,
            # then average over the batch. The negative sign turns maximization into minimization.
            combined_loss = -((per_token_loss * completion_mask).sum(dim=1) / completion_mask.sum(dim=1)).mean()

            # Save scalar values for logging before clearing.
            combined_loss_val = combined_loss.mean().item()
            ppo_loss_val=surrogate_loss.mean().item()
            kl_div_val = kl_div.mean().item()
            print("Final Loss:", combined_loss_val)

            # Remove now-unused intermediate tensors.
            unclipped_objective = None
            clipped_ratio = None
            clipped_objective = None
            model_log_probs = None
            ref_log_probs = None
            combined_loss = None
            surrogate_loss = None
            A_hat = None
            probability_ratio = None

        # 5. Backpropagation and parameter update (only if not in validation mode).
        is_param_updated = False
        if not is_validation:
            if global_step < warm_up_step:
                optimizer.zero_grad()  # Ensure gradients are zeroed in warm-up.
                scaler.scale(combined_loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                scaler.scale(combined_loss).backward()
                if (step + 1) % gradient_accumulation_step == 0 or (step + 1) == len(dataloader):
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
                    is_param_updated = True

        running_loss += combined_loss.item()

        # 6. Logging with dynamic log group.
        print_step("6. Logging")
        writer.add_scalar(f"{log_group}/combined_loss", combined_loss_val, global_step)
        writer.add_scalar(f"{log_group}/ppo_loss", ppo_loss_val, global_step)
        writer.add_scalar(f"{log_group}/kl_div", kl_div_val, global_step)
        if is_param_updated:
            writer.add_scalar(f"{log_group}/model_update_combined_loss", combined_loss_val, global_step)

        # Remove unused variables from the current iteration.
        completion_logits = None
        completion_ids = None
        ref_completion_logits = None  # Already cleared above if not needed.
        combined_loss = None
        kl_div = None

        print_step("7. End Loop")
        global_step += 1

    print_step("_.1. run() exit")
    return running_loss, global_step



def train_and_evaluate(model, ref_model, dataloader, val_dataloader, optimizer, device, num_epochs, group_size,
                       num_grpo, epsilon, kl_lambda, scaler, save_epochs, start_epoch,
                       warming_up_step, gradient_accumulation_step, skip_validation_step=False, 
                       temperature=1.0):
    writer = SummaryWriter(log_dir="runs/starcoder2_reasoning")

    # --- Generate sample output text after each epoch ---
    model.eval()  # Set to eval mode for generation.
    with torch.no_grad():
        samping(model, tokenizer, device, 0, writer,
                "In Custom Clang-repl, What is the prompt in Custom Clang-repl?",
                "```\n>>> (prompt)\n```")
        samping(model, tokenizer, device, 0, writer,
                "In Custom Clang-repl, Do we allow multiline comments or backslash-extended lines in Custom Clang-repl Test?",
                "Custom Clang-repl takes only one line input.")
    model.train()  # Switch back to training mode.

    global_step = 0

    for epoch in range(start_epoch, num_epochs):
        running_loss = 0.0
        print_step(f"Epoch {epoch+1}/{num_epochs} - Validation", main_step=True)

        if not skip_validation_step:
            print_step("_. Validation")
            print_memory(20)
            _old_model = copy.deepcopy(model).half()
            _old_model.eval()
            for param in _old_model.parameters():
                param.requires_grad = False
            print_memory(21)
            
            # Run validation (no parameter updates).
            _, _ = run(model, _old_model, ref_model, val_dataloader, optimizer, device, tokenizer,
                                 group_size, epsilon, kl_lambda, scaler, writer, global_step,
                                 log_group="val_Loss", warm_up_step=warming_up_step,
                                 gradient_accumulation_step=gradient_accumulation_step, is_validation=True,
                                 temperature=temperature)
            _old_model = None

        # Loop over gradient groups for training.
        for grpo_idx in range(num_grpo):
            print_step(f"Epoch {epoch+1}/{num_epochs} - Training Gradient Group {grpo_idx+1}/{num_grpo}", main_step=True)
            print_memory(20)
            old_model = copy.deepcopy(model).half()
            old_model.eval()
            for param in old_model.parameters():
                param.requires_grad = False
            print_memory(21)

            loss, global_step = run(model, old_model, ref_model, dataloader, optimizer, device, tokenizer,
                                      group_size, epsilon, kl_lambda, scaler, writer, global_step,
                                      log_group="Loss", warm_up_step=warming_up_step,
                                      gradient_accumulation_step=gradient_accumulation_step, is_validation=False,
                                    temperature=temperature)
            running_loss += loss
            old_model = None

        print_step("7. End Epoch")
        avg_loss = running_loss / len(dataloader)
        if log_step: print(f"Epoch {epoch + 1} completed. Average Loss: {avg_loss:.4f}")
        writer.add_scalar("Epoch/Average_Loss", avg_loss, epoch + 1)

        # Save latest checkpoint.
        checkpoint = {
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'epoch': epoch + 1
        }
        os.makedirs(os.path.dirname(last_checkpoint_path), exist_ok=True)
        torch.save(checkpoint, last_checkpoint_path)

        # Optionally save checkpoint on specific epochs.
        if save_epochs is not None and epoch % save_epochs == 0:
            checkpoint_dir = checkpoint_dir_pre + str(epoch + 1)
            os.makedirs(checkpoint_dir, exist_ok=True)
            checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.pt")
            torch.save(checkpoint, checkpoint_path)
            if log_step: print(f"Checkpoint saved at epoch {epoch + 1} to {checkpoint_path}")

            # Save tokenizer once if not already saved.
            tokenizer_save_dir = "./saved_models/tokenizer"
            if not os.path.exists(tokenizer_save_dir):
                os.makedirs(tokenizer_save_dir, exist_ok=True)
                tokenizer.save_pretrained(tokenizer_save_dir)
                if log_step: print("Tokenizer saved.")


    writer.close()
    return avg_loss

def train(
        num_epochs,
        lr,
        kl_lambda,
        epsilon,
        num_grpo,
        group_size,
        warming_up_step,
        gradient_accumulation_step,
        save_epochs=None,
        skip_validation_step=False,
        is_finding_opt=False,
        temperature=1.0):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Check if a latest checkpoint exists to load model and optimizer states
    if os.path.exists(last_checkpoint_path) and not is_finding_opt:
        print_memory(1)
        checkpoint = torch.load(last_checkpoint_path, map_location=torch.device("cpu"))
        _model = AutoModelForCausalLM.from_pretrained(model_id)
        config = copy.deepcopy(_model.config)
        _model = None
        print_memory(2)
        config.num_hidden_layers += 2
        model = AutoModelForCausalLM.from_config(config)
        print_memory(3)
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(device)
        print_memory(4)
        optimizer = Adafactor(get_layer_params(model), lr=lr, relative_step=False, scale_parameter=False)
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint.get('epoch', 0)
        if log_step: print(f"Loaded checkpoint from {ref_checkpoint_path} at epoch {start_epoch}")
        print_memory(7)
    else:
        if os.path.exists(ref_checkpoint_path):
            print_memory(1)
            checkpoint = torch.load(ref_checkpoint_path, map_location=torch.device("cpu"))
            _model = AutoModelForCausalLM.from_pretrained(model_id)
            config = copy.deepcopy(_model.config)
            _model = None
            print_memory(2)
            config.num_hidden_layers += 2
            model = AutoModelForCausalLM.from_config(config)
            print_memory(3)
            model.load_state_dict(checkpoint['model_state_dict'])
            model.to(device)
            print_memory(4)
            optimizer = Adafactor(get_layer_params(model), lr=lr, relative_step=False, scale_parameter=False)
            #optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = 0 #checkpoint.get('epoch', 0)
            if log_step: print(f"Loaded checkpoint from {ref_checkpoint_path} at epoch {start_epoch}")
            print_memory(7)

        else:
            assert False, "prompt_last_checkpoint_path must exist"

    dups = check_optimizer_duplicates(optimizer)
    if dups:
        print("Warning: The optimizer contains duplicate parameters!")
        print(f"Duplicate parameter count: {len(dups)}")
    else:
        print("No duplicate parameters found in the optimizer.")

    # Clear cached memory that is no longer used
    torch.cuda.empty_cache()
    gc.collect()
    print_memory(9)

    # Reference model (for KL)
    old_model = None
    ref_model = copy.deepcopy(model).half().eval()
    for param in ref_model.parameters():
        param.requires_grad = False
    print_memory(10)
    
    # DataLoader
    global batch_size
    dataloader = DataLoader(
        tokenized_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=data_collator
    )
    val_dataloader = DataLoader(
        val_tokenized_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=data_collator
    )

    # AMP GradScaler
    scaler = torch.cuda.amp.GradScaler()

    # Train & get final metric
    final_avg_loss = train_and_evaluate(
        model=model,
        ref_model=ref_model,
        dataloader=dataloader,
        val_dataloader=val_dataloader,
        optimizer=optimizer,
        device=device,
        num_epochs=num_epochs,
        group_size=group_size,
        num_grpo=num_grpo,
        epsilon=epsilon,
        kl_lambda=kl_lambda,
        scaler=scaler,
        save_epochs=save_epochs,
        start_epoch=start_epoch,
        warming_up_step=warming_up_step,
        gradient_accumulation_step=gradient_accumulation_step,
        skip_validation_step=skip_validation_step,
        temperature=temperature
        
    )

    # Return the final average loss to Optuna
    return final_avg_loss


# ------------------------------------------------
# Optuna Objective Function
# ------------------------------------------------
def objective(trial):
    """
    Defines how Optuna will run each trial:
    - sample hyperparameters
    - set up the model & optimizer with those
    - run a short training loop
    - return a metric (the final avg loss) to minimize
    """
    global group_size
    num_epochs, lr, kl_lambda, epsilon, num_grpo, warming_up_step, gradient_accumulation_step, temperature = object_hiper_param(trial)

    print(
        f"[Optuna] Trial hyperparameters -> lr: {lr}, kl_lambda: {kl_lambda}, epsilon: {epsilon}, num_grpo: {num_grpo}, warming_up_step: {warming_up_step}, gradient_accumulation_step: {gradient_accumulation_step}, temperature: {temperature}")
    return train(
        num_epochs=num_epochs,
        lr=lr,
        kl_lambda=kl_lambda,
        epsilon=epsilon,
        num_grpo=num_grpo,
        group_size=group_size,
        warming_up_step=warming_up_step, 
        gradient_accumulation_step=gradient_accumulation_step,
        skip_validation_step=True,
        is_finding_opt=True,
        temperature=temperature
    )


# ------------------------------------------------
# Run Optuna Study
# ------------------------------------------------
if __name__ == "__main__":
    if is_finding_opt:
        # Create study to minimize final loss
        study = optuna.create_study(direction="minimize")
        study.optimize(objective, n_trials=5)  # You can increase n_trials

        print("Study completed!")
        print("Best trial:")
        best_trial = study.best_trial
        print(f"  Value: {best_trial.value}")
        print("  Params: ")
        for key, value in best_trial.params.items():
            print(f"#    {key}: {value}")
        with open("hiper_param.json", "w") as f:
            json.dump(best_trial.params.items(), f, indent=4)
        # Study completed!
        # Best trial:
        #  Value: 715.3611988491482
        #  Params:
        #    lr: 0.0002746775018590349
        #    kl_lambda: 0.10527608699361579
        #    epsilon: 0.12442505216944565
        #    num_grpo: 2
    else:
        train(
            num_epochs=num_epochs,
            lr=lr,
            kl_lambda=kl_lambda,
            epsilon=epsilon,
            num_grpo=num_grpo,
            group_size=group_size,
            save_epochs=save_epochs,
            warming_up_step=warming_up_step,
            gradient_accumulation_step=gradient_accumulation_step,
            skip_validation_step=False,
            temperature=temperature
        )


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

[I 2025-03-25 23:40:56,038] A new study created in memory with name: no-name-0203ce69-423b-4cc5-9a77-8cc40d9eb2af


[Optuna] Trial hyperparameters -> lr: 1.5069929525878771e-05, kl_lambda: 0.02398748051937347, epsilon: 0.19806932805426544, num_grpo: 2, warming_up_step: 1, gradient_accumulation_step: 5, temperature: 0.5959953460059638
No duplicate parameters found in the optimizer.
Epoch 1/1 - Validation
Epoch 1/1 - Training Gradient Group 1/2


  0%|                                                                                                                                              | 0/10 [00:00<?, ?it/s]

...

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌             | 9/10 [07:31<00:49, 49.58s/it]

std_rewards: 0.0 Rewards: [0, 0, 0, 0]
Final Loss: nan


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [08:22<00:00, 50.24s/it]
[W 2025-03-26 00:40:20,270] Trial 3 failed with parameters: {'lr': 2.858527077139412e-06, 'kl_lambda': 0.05757740368847213, 'epsilon': 0.10866377220354449, 'num_grpo': 1, 'warming_up_step': 1, 'gradient_accumulation_step': 4, 'temperature': 0.5323536191000062} because of the following error: The value nan is not acceptable.
[W 2025-03-26 00:40:20,271] Trial 3 failed with value nan.


[Optuna] Trial hyperparameters -> lr: 1.1269427459439703e-06, kl_lambda: 0.07066949695746146, epsilon: 0.15373160925195395, num_grpo: 1, warming_up_step: 1, gradient_accumulation_step: 3, temperature: 1.1301817108632517
No duplicate parameters found in the optimizer.
Epoch 1/1 - Validation
Epoch 1/1 - Training Gradient Group 1/1


  0%|                                                                                                                                              | 0/10 [00:00<?, ?it/s]